# Guide-net

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## Loading the data

In [1]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.datasets import cifar10

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=1974)

In [0]:
train_features = X_train.reshape(X_train.shape[0], 32, 32, 3)/255
vali_features = X_val.reshape(X_val.shape[0], 32, 32, 3)/255
test_features = X_test.reshape(X_test.shape[0], 32, 32, 3)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

## Defining coarse labels

In [0]:
dict_coarse1 = {0: 0,
              1: 0,
              2: 1,
              3: 1,
              4: 1,
              5: 1,
              6: 1,
              7: 1,
              8: 0,
              9: 0}

dict_coarse2 = {0: 0,
              1: 1,
              2: 3,
              3: 4,
              4: 6,
              5: 4,
              6: 5,
              7: 6,
              8: 2,
              9: 1}

y_train_coarse1 = np.vectorize(dict_coarse1.get)(y_train)
y_val_coarse1 = np.vectorize(dict_coarse1.get)(y_val)
y_test_coarse1 = np.vectorize(dict_coarse1.get)(y_test)

y_train_coarse2 = np.vectorize(dict_coarse2.get)(y_train)
y_val_coarse2 = np.vectorize(dict_coarse2.get)(y_val)
y_test_coarse2 = np.vectorize(dict_coarse2.get)(y_test)

y_train_c_cat1 = np_utils.to_categorical(y_train_coarse1)
y_val_c_cat1 = np_utils.to_categorical(y_val_coarse1)
y_test_c_cat1 = np_utils.to_categorical(y_test_coarse1)

y_train_c_cat2 = np_utils.to_categorical(y_train_coarse2)
y_val_c_cat2 = np_utils.to_categorical(y_val_coarse2)
y_test_c_cat2 = np_utils.to_categorical(y_test_coarse2)

## Training the model

In [0]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [0]:
class TimingCallback(keras.callbacks.Callback):
  """Callback that saves the time elapsed of each epoch to the log.
  """  
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [0]:
class LossWeightsModifier3Vars(keras.callbacks.Callback):
  """Callback that modifies the loss weights on certain epochs.
    # Arguments
        alpha: coeficient for the coarser class.
        beta: coeficient for the second level coarse class.
        gamma: coeficient for the fine class.
  """  
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
    
  def on_epoch_end(self, epoch, logs={}):

    if epoch == 4:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.2)
      print('Changing loss weights to: coarse1 = {}, coarse2 = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta), K.eval(self.gamma)))
    
    if epoch == 14:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0.4)
      K.set_value(self.gamma, 0.6)  
      print('Changing loss weights to: coarse1 = {}, coarse2 = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta), K.eval(self.gamma)))
  
    if epoch == 21:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)  
      print('Changing loss weights to: coarse1 = {}, coarse2 = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta), K.eval(self.gamma)))
  
    logs['alpha'] = K.eval(self.alpha)
    logs['beta'] = K.eval(self.beta) 
    logs['gamma'] = K.eval(self.gamma) 

In [0]:
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)

num_classes_coarse1 = 2
num_classes_coarse2 = 7
num_classes_fine = 10

img_input = Input(shape=input_shape, name='input')

# coarse1 network
coarse1 = Conv2D(128, (3, 3), activation='relu', name='c1_block1_conv1', padding='same')(img_input)

coarse1 = Conv2D(128, (3, 3), activation='relu', name='c1_block1_conv2', padding='same')(coarse1)
coarse1 = MaxPooling2D((2, 2), name='c1_block2_pool')(coarse1)

coarse1 = Conv2D(128, (3, 3), activation='relu', name='c1_block1_conv3', padding='same')(coarse1)
coarse1 = MaxPooling2D((2, 2), name='c1_block3_pool')(coarse1)

coarse1 = Flatten(name='c1_flatten')(coarse1)
coarse1 = Dense(512, name='c1_fc_1')(coarse1)
coarse1 = BatchNormalization()(coarse1)
coarse1 = Activation('relu')(coarse1)
coarse1 = Dropout(0.5)(coarse1)

coarse1 = Dense(512, name='c1_fc_2')(coarse1)
coarse1 = BatchNormalization()(coarse1)
coarse1 = Activation('relu')(coarse1)
coarse1 = Dropout(0.5)(coarse1)


# coarse2 network
coarse2 = Conv2D(128, (3, 3), activation='relu', name='c2_block1_conv1', padding='same')(img_input)

coarse2 = Conv2D(128, (3, 3), activation='relu', name='c2_block1_conv2', padding='same')(coarse2)
coarse2 = MaxPooling2D((2, 2), name='c2_block2_pool')(coarse2)

coarse2 = Conv2D(128, (3, 3), activation='relu', name='c2_block1_conv3', padding='same')(coarse2)
coarse2 = MaxPooling2D((2, 2), name='c2_block3_pool')(coarse2)

coarse2 = Flatten(name='c2_flatten')(coarse2)
coarse2 = Dense(512, name='c2_fc_1')(coarse2)
coarse2 = BatchNormalization()(coarse2)
coarse2 = Activation('relu')(coarse2)
coarse2 = Dropout(0.5)(coarse2)

coarse2 = Dense(512, name='c2_fc_2')(coarse2)
coarse2 = BatchNormalization()(coarse2)
coarse2 = Activation('relu')(coarse2)
coarse2 = Dropout(0.5)(coarse2)

coarse2 = keras.layers.concatenate([coarse2, coarse1])

# fine network
fine = Conv2D(128, (3, 3), activation='relu', name='block1_conv1', padding='same')(img_input)

fine = Conv2D(128, (3, 3), activation='relu', name='block1_conv2', padding='same')(fine)
fine = MaxPooling2D((2, 2), name='block2_pool')(fine)

fine = Conv2D(128, (3, 3), activation='relu', name='block1_conv3', padding='same')(fine)
fine = MaxPooling2D((2, 2), name='block3_pool')(fine)

fine = Flatten(name='flatten')(fine)
fine = Dense(512, name='fc_1')(fine)
fine = BatchNormalization()(fine)
fine = Activation('relu')(fine)
fine = Dropout(0.5)(fine)

fine = Dense(512, name='fc_2')(fine)
fine = BatchNormalization()(fine)
fine = Activation('relu')(fine)
fine = Dropout(0.5)(fine)

fine = keras.layers.concatenate([fine, coarse2])

coarse_pred1 = Dense(num_classes_coarse1, activation='softmax', name='predictions_coarse1')(coarse1)
coarse_pred2 = Dense(num_classes_coarse2, activation='softmax', name='predictions_coarse2')(coarse2)
fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(fine)

In [17]:
model = Model(inputs=img_input, outputs= [coarse_pred1, coarse_pred2, fine_pred], name='guide_net')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
c1_block1_conv1 (Conv2D)        (None, 32, 32, 128)  3584        input[0][0]                      
__________________________________________________________________________________________________
c2_block1_conv1 (Conv2D)        (None, 32, 32, 128)  3584        input[0][0]                      
__________________________________________________________________________________________________
c1_block1_conv2 (Conv2D)        (None, 32, 32, 128)  147584      c1_block1_conv1[0][0]            
__________________________________________________________________________________________________
c2_block1_

In [0]:
alpha = K.variable(value=0.9, dtype="float32", name="alpha") 
beta = K.variable(value=0.1, dtype="float32", name="beta") 
gamma = K.variable(value=0, dtype="float32", name="gamma") 

adam = optimizers.Adam(lr=0.00025)

model.compile(loss='categorical_crossentropy', 
              optimizer=adam,
              loss_weights=[alpha, beta, gamma],
              metrics=['accuracy'])

In [0]:
clr_cb = ReduceLROnPlateau(monitor='val_predictions_fine_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
change_lw = LossWeightsModifier3Vars(alpha, beta, gamma)
csv_cb = CSVLogger('drive/TCC-ITAU/cifar-10/training-data/guide-net/guide-net.csv', separator=',', append=False)


cbks = [clr_cb, time_cb, change_lw, csv_cb]

In [20]:
epochs = 50
start_time = time()
training = model.fit(train_features, [y_train_c_cat1, y_train_c_cat2, y_train_cat],
                     validation_data=(vali_features, [y_val_c_cat1, y_val_c_cat2, y_val_cat]),
                     epochs=epochs,
                     batch_size=256, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 69s 2ms/step - loss: 0.4742 - predictions_coarse1_loss: 0.3658 - predictions_coarse2_loss: 1.4499 - predictions_fine_loss: 2.9893 - predictions_coarse1_acc: 0.8570 - predictions_coarse2_acc: 0.4857 - predictions_fine_acc: 0.1091 - val_loss: 0.3891 - val_predictions_coarse1_loss: 0.3137 - val_predictions_coarse2_loss: 1.0680 - val_predictions_fine_loss: 2.4905 - val_predictions_coarse1_acc: 0.8885 - val_predictions_coarse2_acc: 0.6003 - val_predictions_fine_acc: 0.1251
Epoch 2/50
40000/40000 [==============================] - 64s 2ms/step - loss: 0.3178 - predictions_coarse1_loss: 0.2381 - predictions_coarse2_loss: 1.0354 - predictions_fine_loss: 2.9695 - predictions_coarse1_acc: 0.9078 - predictions_coarse2_acc: 0.6251 - predictions_fine_acc: 0.1100 - val_loss: 0.4717 - val_predictions_coarse1_loss: 0.4328 - val_predictions_coarse2_loss: 0.8223 - val_predictions_fine_loss: 2.4248

In [21]:
model.evaluate(test_features, [y_test_c_cat1,y_test_c_cat2, y_test_cat])

10000/10000 [==============================] - 8s 774us/step


[1.1445696430444718,
 0.14849405952841044,
 0.7277668886661529,
 1.1445696430444718,
 0.9438,
 0.8359,
 0.784]

In [22]:
print('Total training time: {}'.format(training_time))

Total training time: 3270.574536561966


In [23]:
model.save('drive/TCC-ITAU/cifar-10/models/guide-net.h5')

TypeError: ignored